In [158]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import widgetbox, layout, row
from bokeh.models.widgets import CheckboxGroup
from astropy.cosmology import FlatLambdaCDM
output_notebook()
%matplotlib notebook

Loading BokehJS ...

In [71]:
hubble_data = np.genfromtxt('hw_2_data/Union2.1RedshiftMag.txt', usecols=(0,1,2,3),
                            names='name, redshift, distance_mod, distance_mod_err',
                            dtype=['O', 'f8', 'f8', 'f8'])
hubble_data = pd.DataFrame(hubble_data)
sample_info = np.genfromtxt('hw_2_data/SCPUnion2.1_AllSNe.txt', delimiter=' & ', usecols=(0,7),
                            names='name, sample_num', dtype=['S12','int'])
sample_info = pd.DataFrame(sample_info)
sample_dict = {1: 'Hamuy et al. (1996)',
               2: 'Krisciunases et al. (2005)',
               3: 'Riess et al. (1999)',
               4: 'Jha et al. (2006)',
               5: 'Kowalski et al. (2008)',
               6: 'Hicken et al. (2009)', 
               7: 'Contreras et al. (2010)',
               8: 'Holtzman et al. (2009)',
               9: 'Riess et al. (1998)',
               10: 'Perlmutter et al. (1999)',
               11: 'Barris et al. (2004)',
               12: 'Amanullah et al. (2008)',
               13: 'Knop et al. (2003)',
               14: 'Astier et al. (2006)',
               15: 'Miknaitis et al. (2007)',
               16: 'Tonry et al. (2003)',
               17: 'Riess et al. (2007)',
               18: 'Amanullah et al. (2010)',
               19: 'Suzuki et al. (2012)'}
sample_info['sample'] = pd.Series([sample_dict[i] for i in sample_info['sample_num']])

merged_data = pd.merge(hubble_data, sample_info)

In [108]:
colors = ['#a09336',
          '#b253bf',
          '#60b544',
          '#6667c9',
          '#b6ba38',
          '#d54b94',
          '#51bc7c',
          '#cf4156',
          '#4bb8aa',
          '#cb4626',
          '#5d92cd',
          '#db9a47',
          '#bc88d1',
          '#477a3c',
          '#9b496e',
          '#9fb36a',
          '#de8294',
          '#846d30',
          '#e57d4c',
          '#a75c38']
merged_data['color'] = pd.Series([colors[n] for n in merged_data['sample_num']])

In [117]:
def errorbar(fig, x, y, xerr=None, yerr=None, color='b', point_kwargs={}, error_kwargs={}):
    # Bokeh doesn't have error bars built-in!
    # This function comes from a StackOverflow answer by user MaxNoe
    # http://stackoverflow.com/questions/29166353/how-do-you-add-error-bars-to-bokeh-plots-in-python
    fig.circle(x, y, color=color, alpha=0.8, **point_kwargs)

    if xerr is not None:
        x_err_x = []
        x_err_y = []
        for px, py, err in zip(x, y, xerr):
            x_err_x.append((px - err, px + err))
            x_err_y.append((py, py))
        fig.multi_line(x_err_x, x_err_y, color=color, alpha=0.8, **error_kwargs)

    if yerr is not None:
        y_err_x = []
        y_err_y = []
        for px, py, err in zip(x, y, yerr):
            y_err_x.append((px, px))
            y_err_y.append((py - err, py + err))
        fig.multi_line(y_err_x, y_err_y, color=color, alpha=0.8, **error_kwargs)

In [162]:
cosmo = FlatLambdaCDM(Om0=1-0.729, H0=70)
z = np.linspace(min(merged_data['redshift']), max(merged_data['redshift']), 1000)
distmods = cosmo.distmod(z=z).value

In [163]:
x = []
def sample_selection(attr, old, new):
    x.append((attr, old, new))

In [164]:
TOOLS = 'wheel_zoom,box_zoom,reset'

p = figure(title='Union 2.1 Compilation Hubble Diagram', tools=TOOLS)
errorbar(p, merged_data['redshift'], merged_data['distance_mod'], 
         yerr=merged_data['distance_mod_err'], color=merged_data['color'])
# p.line(z, distmods, color='black', line_width=2, alpha=0.5)

selection = CheckboxGroup(active=[], labels=list(merged_data['sample'].unique()))
selection.on_change('active', sample_selection)
s = show(row(p, widgetbox(selection)), notebook_handle=True)

NotImplementedError: cannot make a list of Quantities.  Get list of values with q.value.list()

In [156]:
s

NameError: name 's' is not defined